In [ ]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append("../src")
from aggregated_models.myimports  import *
from aggregated_models import myJupyterUtils ## Remove stacktraces on Keyboardinterupt
plt.style.use('ggplot')

# helpers to compute metrics
from aggregated_models.validation import MetricsComputer,  LLH  
Validation = MetricsComputer("click")

# baselines
from aggregated_models.basicmodels import LogisticModel, NaiveBayesModel, LogisticModelWithCF 
from aggregated_models.aggLogistic import AggLogistic

# loading public "criteo attribution dataset"
from aggregated_models import loaddata  

In [ ]:
# code to prepare the aggregated dataset
from aggregated_models.aggdataset import AggDataset 

In [ ]:
## Most relevant code is there:
from aggregated_models.agg_mrf_model import AggMRFModel
import aggregated_models.agg_mrf_model
# also in https://gitlab.criteois.com/a.gilotte/aggdata/-/blob/master/src/baseaggmodel.py

## Download Data
- downloading criteo-research-attribution-dataset
- from url http://go.criteo.net

In [ ]:
#loaddata.download_dataset()

## Loading data
3 versions of the dataset are used for experiments: "small" , "sampled" and "full"
- "full" has 11 features with about 16M samples
- "sampled" has the same 11 features, but only 160k samples
- "small" also has 160k samples, but only the 5 features with lowest modalities count, and allow for fast experiments.

In [ ]:
dataset = "small" # fast expriments
# dataset= "sampled" # Training a MRF may require 5h and 16Go data
# dataset= "full"  # Training a MRF may require 32Go, and several days

In [ ]:
train, valid, features, label = loaddata.getDataset(dataset)

In [ ]:
train.sample(2)

In [ ]:
for f in features:
    nbModalities = len(set(train[f].values))
    print( f"feature {f} has {nbModalities} distinct modalities" )

## Preparing Aggregated data

- aggdata contains projections of number of displays and clicks along each pair of feature
- may also add some noise to make it differential private
- the goal is to learn a model predicting Proba( label | features) using *only* those aggdata.

In [ ]:
# parameters for of the privacy protecting noise.
epsilon = None  # Set to None to get no noise.
delta = None 

In [ ]:
aggdata = AggDataset( train , features, "*&*",  label, epsilon, delta )

#https://gitlab.criteois.com/a.gilotte/aggdata/-/blob/master/src/featuremappings.py#L205

In [ ]:
print( f" Nb Queries: {len(aggdata.aggDisplays)}")
print( f" Noise distribution: {aggdata.noiseDistribution}" )

In [ ]:
# aggdata may be viewed as a dictionary queryname -> dataframe
aggdata_datframe_dico = aggdata.toDFs()
queries = [x for x in aggdata_datframe_dico.keys()]
print( f"list of queries {queries}" )

In [ ]:
# Dataframe for the query  " select 'cat1', 'cat8' , count, sum(label) group by 'cat1', 'cat8' "
aggdata_datframe_dico["cat1&cat8"].sample(3)

In [ ]:
aggdata.aggDisplays

# Dictionary of projections 

In [ ]:
aggdata.aggDisplays["cat8&cat9"]

In [ ]:
aggdata.aggDisplays["cat8"].Data

# a "projection" contains counts stored in an array. 
# there is dictionary modality -> index in array

In [ ]:
# dico is stored in this class:
aggdata.aggDisplays["cat8&cat9"].feature
##https://gitlab.criteois.com/a.gilotte/aggdata/-/blob/master/src/featuremappings.py#L32

# Each modality of "cat8" found in train was assigned an id, from 0 to NbCat8-1.  
# At index NbCat8, it is the count for the modality " Not found in train".  (But maybe in valid )

In [ ]:
df = train.sample(4).copy()
## changing initial modality by index
aggdata.aggDisplays["cat8"].feature.Map( df ) ## replacing modalities of cat8 by modalities from 1 to NbCat8
aggdata.aggDisplays["cat9"].feature.Map( df )

aggdata.aggDisplays["cat8&cat9"].feature.Map( df ) ##  cat8&cat9 = cat8 + nbCat8 * cat9    (Or is i the opposite ?)

##### logistic Regression
- Using full log instead of aggdata. 
- Training with all "crossfeatures" found in agg data ( ie quadratic kernell)
- We do not expect to do better, the goal is to get similar performances

In [ ]:
regulL2 = 16
logisticCfs = LogisticModelWithCF( "click" , features, "*&*"  , train ,
                                      hashspace=2**22 , lambdaL2 = regulL2  )
logisticCfs.fit( train )
print( f"Logistic(*&*), l2:{regulL2}" ,  "train",  Validation.run(logisticCfs,train) , "valid" , Validation.run(logisticCfs,valid)   )

##### logistic Regression from aggregated clicks and full display data (quadratic kernell)
 - same performances as "standard" logistic regression
 - but still using full display data, so not really usefull

In [ ]:
    regulL2 = 16
    logisticCfs = AggLogistic(  aggdata , features, clicksCfs = "*&*" , regulL2=regulL2 )
    logisticCfs.fit( train[features] , nbIter = 200 )
    print( f"Logistic(*&*), l2:{regulL2}" ,  "train",  Validation.run(logisticCfs,train) , "valid" , Validation.run(logisticCfs,valid)   )

##### Proposed MRF model
- uses only aggregated data
- almost retrieves logitic performances

In [ ]:
regulL2 = 16
nbSamples = 10000
nbIter = 50

self = AggMRFModel( aggdata,
                    features , 
                    exactComputation=False ,  ## Using Gibbs Sampling.  actualy exact=True is broken in latest code
                    clicksCfs = "*&*", ## crossfeatures used by P(Y|X) part of the model
                    displaysCfs="*&*", ## crossfeatures used by P(X) part of the model. Here, all pairs + all single .
                    nbSamples = nbSamples, ## Nb Gibbs samples to estimate gradient
                    regulL2=1.0 ,  ## parmeter "lambda_2"
                    regulL2Click = regulL2  ## parmeter "lambda_1" 
                  )

In [ ]:
self.fit(nbIter)
print( f"MRF lambda1= {regulL2}",  "train",   Validation.run(self,train) , "valid" , Validation.run(self,valid)   )

In [ ]:
# all parameters mu and theta concatenated in a  single vector
self.parameters

# This vector is the concatenation of parameters for associated to each projection

In [ ]:
# List of features and crossfeatures for mu
self.displayWeights

In [ ]:
# List of features and cfs for theta
self.clickWeights
# class WeightsSet : https://gitlab.criteois.com/a.gilotte/aggdata/-/blob/master/src/baseaggmodel.py#L8

## In parameter vector, indices from 3719 to 3729 are the parameters "theta" 
##   associated to values of the single feature "cat1"

In [ ]:
# there are also two 'intercept' parameters:
self.muIntercept, self.lambdaIntercept
#  ...  thus P(Y = 1 |X =x) = sigmoid( K(x) . self.parameters[someOffset:] +  self.lambdaIntercept )

#  todo:  remane self.lambdaIntercept to self.thetaIntercept to get coherent notations

In [ ]:
## samples of "X"

self.samples.data.shape

In [ ]:
## Computing dotproducts between K(x) and mu or theta:

## https://gitlab.criteois.com/a.gilotte/aggdata/-/blob/master/src/baseaggmodel.py#L62

mus    = self.dotproducts( self.displayWeights, self.samples.data ) + self.muIntercept
mus

## https://gitlab.criteois.com/a.gilotte/aggdata/-/blob/master/src/agg_mrf_model.py#L145

## note: I just added some comments in the code, translating all the line numbers ...

In [ ]:
d = self.Data
d
#  vector with  the counts of click or display  from aggregated data.
# Same indexing as self.parameters

In [ ]:
p = self.getPredictionsVector( self.samples )
p
# expected counts according to the model, computed by MC on the samples 

In [ ]:
## https://gitlab.criteois.com/a.gilotte/aggdata/-/blob/master/src/agg_mrf_model.py#L187
w = self.displayWeights["cat1"]
w.feature.Project_(  self.samples.data  , self.samples.pdisplays ) # Correct for grads

# a bit uselessly complicated :  self.samples.pdisplays  is constant
# This allows having samples with different 'weights', for example one sample for each possible modality of X

In [ ]:
# After fiting the model,  "data" and "prediction" should be equal
plt.plot( d,p, "x" )


In [ ]:
# ... up to the noise of the sampling / convergence of optimizer
plt.plot( np.log (1+d), np.log( 1+p), "x" )

In [ ]:
## gradient is equal to d-p  + gradient (regularization)
g = self.computeGradient()

# https://gitlab.criteois.com/a.gilotte/aggdata/-/blob/master/src/agg_mrf_model.py#L228

In [ ]:
self.initParameters()
# https://gitlab.criteois.com/a.gilotte/aggdata/-/blob/master/src/agg_mrf_model.py#L90
##  inital values of mu, theta such that:
##  -  P(Y=1|X) 1s flat nd equal to P(Y=1)   (ie all weights = 0 except intercept)
##  -  for each coordinate Xk of X,  P( Xk ) is correct, and Xk indepedent from Xk'
##     ( => set weights of crossfeatures to 0  
##        and initialize weights of single features to log( P_train( Xk =xk ) )

In [ ]:
self.samples

## https://gitlab.criteois.com/a.gilotte/aggdata/-/blob/master/src/SampleSet.py

In [ ]:
self.samples.data.shape

In [ ]:
self.samples.Df().sample(3)

In [ ]:
## fitting the model.  Sligtly simplified (no line search)

# intialiaze
self = AggMRFModel( aggdata,
                    features , 
                    exactComputation=False ,  ## Using Gibbs Sampling.  actualy exact=True is broken in latest code
                    clicksCfs = "*&*", ## crossfeatures used by P(Y|X) part of the model
                    displaysCfs="*&*", ## crossfeatures used by P(X) part of the model. Here, all pairs + all single .
                    nbSamples = nbSamples, ## Nb Gibbs samples to estimate gradient
                    regulL2=1.0 ,  ## parmeter "lambda_2"
                    regulL2Click = regulL2  ## parmeter "lambda_1" 
                  )

## predicting constant CTR => normalize LLH on train  = 0.0 
print( f"MRF lambda1= {regulL2}",  "train",   Validation.run(self,train) , "valid" , Validation.run(self,valid)   )

alpha = 0.05
for i in range(0,50):
    g = self.computeGradient()
    precondG = g / (self.Data+10)  #  diagonal Hessian at origin = self.Data + regul
    self.parameters -= alpha * precondG
    self.update()  ## 
    self.updateSamplesWithGibbs( self.samples , nbsteps=1 )
print( f"MRF lambda1= {regulL2}",  "train",   Validation.run(self,train) , "valid" , Validation.run(self,valid)   )

In [ ]:
## fitting the model.  Sligtly simplified (no line search)

# intialiaze
self = AggMRFModel( aggdata,
                    features , 
                    exactComputation=False ,  ## Using Gibbs Sampling.  actualy exact=True is broken in latest code
                    clicksCfs = "*&*", ## crossfeatures used by P(Y|X) part of the model
                    displaysCfs="*&*", ## crossfeatures used by P(X) part of the model. Here, all pairs + all single .
                    nbSamples = nbSamples, ## Nb Gibbs samples to estimate gradient
                    regulL2=1.0 ,  ## parmeter "lambda_2"
                    regulL2Click = regulL2  ## parmeter "lambda_1" 
                  )


In [ ]:
## gibbs sampling
self.updateSamplesWithGibbs( self.samples , nbsteps=1 )

##https://gitlab.criteois.com/a.gilotte/aggdata/-/blob/master/src/agg_mrf_model.py#L178

## Internal implem of Gibbs sampler

code below is mostly copy-pasted from agg_mrf_model.fastGibbsSample (adapting a few var names) to allow to unroll the loops and see what the variables contain.

Having to use only numpy to get it fast, and ugly because of that.
(And there is a bug. Can you understand what this version of the code does ? )

In [ ]:
rows = self.samples.data[:,:1000].transpose()
rows.shape

y =self.samples.y[ :1000 ]
# 1000 samples of X

In [ ]:
## extracting all relavant data of the model to run Gibbs sampling.
## and exporting it to numpy arrays. ()
exportedDisplayWeights, exportedClickWeights, modalitiesByVarId, paramsVector = self.exportWeightsAll()


In [ ]:
#  running one step of Gibbs on those samples
print( rows[1] )
agg_mrf_model.fastGibbsSample(exportedDisplayWeights, exportedClickWeights, modalitiesByVarId, paramsVector,
                                 rows , 1, y )
print( rows[1] )

# function "fastGibbsSample" is not on a class and takes as input the model in np.array format
# This allows to run sampling in parallel with joblib 

In [ ]:
    ## Internal implem of Gibbs sampling
    # copy-pasted from agg_mrf_model.fastGibbsSample

    
    ## data describing the crossfeatures
    allcoefsv,allcoefsv2, alloffsets, allotherfeatureid = exportedDisplayWeights
    click_allcoefsv,click_allcoefsv2, click_alloffsets, click_allotherfeatureid = exportedClickWeights
    
    x =rows.transpose()
    nbsamples =x.shape[1]


In [ ]:
self.features[3]

In [ ]:
            # For each feature, ressample this feature conditionally to the other 
            #for varId in f:   
            varId = 3 #  let us just fix one feature here   #  resampling self.features[1] -> "cat4"
    
    
    
            ## data describing the different crossfeatures involving varId  in " K(x).mu" 
            ## Those things are arrays, of len the number of crossfeatures involving varId.
            disp_coefsv = allcoefsv[ varId ] 
            disp_coefsv2 = allcoefsv2[ varId ]
            disp_offsets = alloffsets[ varId ]
            disp_otherfeatureid = allotherfeatureid[ varId ]

            ## idem, but crossfeatures in " K(x).theta" part of the model 
            click_coefsv = click_allcoefsv[ varId ]
            click_coefsv2 = click_allcoefsv2[ varId ]
            click_offsets = click_alloffsets[ varId ]
            click_otherfeatureid = click_allotherfeatureid[ varId ]

            # array of possible modalities of varId 
            modalities = modalitiesByVarId[varId]  ## Should be 0,1,2 ... NbModalities
            ## for each modality, compute P( modality | other features ) as exp( dotproduct)
            ## initializing dotproduct
            mus = np.zeros(( nbsamples , len(modalities)))
            lambdas = np.zeros(( nbsamples , len(modalities)))


In [ ]:
                ## Computing the dotproducts    
                #  For each crossfeature containing varId
                #for varJ in np.arange( 0, len(disp_coefsv) ):

                varJ = 1
                # let m a modality of feature varId, and m' a modality of the other feature
                #  Value of the crossfeature is " m *  disp_coefsv[varJ] + m' * disp_coefsv2[varJ]  " 
                # values of m' in the data
                modsJ = x[ disp_otherfeatureid[varJ] ] * disp_coefsv2[varJ] 
                # all possible modality m
                mods = modalities * disp_coefsv[varJ] 
                # Computing crossfeatures
                ## this is a matrix of shape (nbSamples, nbModalities of varId)
                crossmods = np.add.outer(modsJ , mods) + disp_offsets[varJ]  
                # Adding crossfeature weight.
                mus += paramsVector[crossmods]


In [ ]:
print(crossmods.shape)
crossmods

In [ ]:
            mus = np.exp(mus)
            lambdas = np.exp(lambdas)



In [ ]:
.shape

In [ ]:
len(self.samples.y)